# Project Setup

In [ ]:
import pandas as pd
import numpy as np
import re

# Dataset Analysis
In this Jupyter notebook, we'll analyze the data that we extracted previously.

In [ ]:
DRIVE = 'drive/MyDrive/'
DATASET_NAME = 'combined_dataset.csv'
DATASET_PATH = DRIVE + DATASET_NAME

df = pd.read_csv(DATASET_PATH)
df.head()

,rating,parent_asin,timestamp,review,category
0,5,B01BRVADG6,2019-02-07 22:41:27.737,One of my favorite oils. This oil is wonderful...,All_Beauty
1,5,B01BRVADG6,2017-06-04 13:07:28.000,Five Stars. I love the oils! My skin is so muc...,All_Beauty
2,5,B01BRVADG6,2019-10-16 15:50:59.484,best massage oil. I have now used this product...,All_Beauty
3,5,B01BRVADG6,2017-08-09 11:35:37.781,Five Stars. 2 or 3 time buying I love this oil,All_Beauty
4,5,B01BRVADG6,2017-01-31 12:32:56.000,Five Stars. Awesome,All_Beauty


## Number of reviews

In [ ]:
df.shape

(10514, 5)

In [ ]:
df.groupby(by="category").size()

,0
category,
All_Beauty,391
Amazon_Fashion,346
Appliances,551
Arts_Crafts_and_Sewing,282
Automotive,250
Baby_Products,250
Beauty_and_Personal_Care,250
Books,251
CDs_and_Vinyl,253


In [ ]:
asin_counts = df.groupby(['category', 'parent_asin']).size().reset_index(name='review_count')

asin_review_stats = asin_counts.groupby('category').agg(
    avg_reviews_per_asin=('review_count', 'mean'),
    min_reviews_per_asin=('review_count', 'min'),
    max_reviews_per_asin=('review_count', 'max')
).reset_index()

asin_review_stats

,category,avg_reviews_per_asin,min_reviews_per_asin,max_reviews_per_asin
0,All_Beauty,65.166667,23,208
1,Amazon_Fashion,49.428571,28,103
2,Appliances,91.833333,30,333
3,Arts_Crafts_and_Sewing,56.400000,21,161
4,Automotive,20.833333,9,56
5,Baby_Products,20.833333,1,92
6,Beauty_and_Personal_Care,20.833333,1,107
7,Books,20.916667,5,72
8,CDs_and_Vinyl,21.083333,1,129
9,Cell_Phones_and_Accessories,22.636364,2,192


## Rating

In [ ]:
df['rating'].mean(), df['rating'].std()

(4.264124025109378, 1.2665916872698675)

In [ ]:
df.groupby(by="category").agg({'rating':['mean', 'std']})

rating          
                                 mean       std
category                                       
All_Beauty                   4.084399  1.328403
Amazon_Fashion               3.991329  1.406995
Appliances                   4.382940  1.270790
Arts_Crafts_and_Sewing       4.400709  1.186700
Automotive                   4.212000  1.388079
Baby_Products                4.296000  1.273921
Beauty_and_Personal_Care     3.808000  1.521973
Books                        4.454183  1.054937
CDs_and_Vinyl                4.758893  0.667537
Cell_Phones_and_Accessories  4.285141  1.235815
Clothing_Shoes_and_Jewelry   4.339921  1.216237
Digital_Music                4.836653  0.482919
Electronics                  4.374502  1.157233
Gift_Cards                   4.560311  1.134190
Grocery_and_Gourmet_Food     4.084000  1.372764
Handmade_Products            4.657480  0.873865
Health_and_Household         4.348000  1.262248
Health_and_Personal_Care     3.715415  1.625088
Home_and_Kitchen             4.434263  1.175866
Industrial_and_Scientific    4.052000  1.388889
Kindle_Store                 4.254032  1.058981
Magazine_Subscriptions       4.100372  1.387968
Movies_and_TV                3.808000  1.412486
Musical_Instruments          4.508000  0.906212
Office_Products              4.306773  1.282776
Patio_Lawn_and_Garden        4.236000  1.303495
Pet_Supplies                 4.159363  1.350001
Software                     4.095238  1.296201
Sports_and_Outdoors          4.253012  1.268509
Subscription_Boxes           4.025330  1.408909
Tools_and_Home_Improvement   4.403226  1.112457
Toys_and_Games               4.514056  1.047656
Unknown                      4.374498  1.176534
Video_Games                  4.208000  1.227503

## Review length

In [ ]:
df['num_characters'] = df['review'].apply(len)
df['num_words'] = df['review'].apply(lambda x: len(x.split()))

df['num_characters'].mean(), df['num_words'].mean()

(228.62830511698687, 42.0527867605098)

In [ ]:
result = df.groupby('category').agg(
    avg_characters=('num_characters', 'mean'),
    min_characters=('num_characters', 'min'),
    max_characters=('num_characters', 'max'),
    avg_words=('num_words', 'mean'),
    min_words=('num_words', 'min'),
    max_words=('num_words', 'max')
).reset_index()

result

,category,avg_characters,min_characters,max_characters,avg_words,min_words,max_words
0,All_Beauty,197.040921,8,2349,36.815857,2,456
1,Amazon_Fashion,168.462428,8,1552,31.141618,2,300
2,Appliances,145.568058,5,1541,26.381125,2,273
3,Arts_Crafts_and_Sewing,149.319149,6,1495,27.521277,2,298
4,Automotive,219.672000,10,2062,40.436000,2,362
5,Baby_Products,204.328000,12,1674,37.532000,2,301
6,Beauty_and_Personal_Care,235.448000,10,1539,44.580000,2,281
7,Books,318.474104,10,4388,57.478088,2,760
8,CDs_and_Vinyl,175.549407,4,2392,32.715415,2,395
9,Cell_Phones_and_Accessories,199.224900,14,1586,37.646586,3,310
